In [1]:
import sys
import os
sys.path.insert(0, './magcolloids')
sys.path.insert(0, './confined-colloids')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML, clear_output
import jsonpickle
import scipy.optimize as spo
import scipy.spatial as spa 
import magcolloid as mgc

import lammps2d as lmp

ureg = mgc.ureg

idx = pd.IndexSlice

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
region, initial_positions = lmp.initial_setup(n_of_particles = 150, packing=0.35, height = 4.1)

particles = mgc.particles(
    initial_positions*ureg.um,
    radius = 1.4*ureg.um,
    susceptibility = 0.4,
    diffusion=0.073*ureg.um**2/ureg.s,
    density = 1000*ureg.kg/ureg.m**3,
    temperature=300*ureg.K)

In [3]:
field = mgc.field(magnitude = 7.28*ureg.mT, frequency = 0*ureg.Hz, angle = 0*ureg.degrees,multibody_iter=2)

In [4]:
world = mgc.world(particles, temperature = 300*ureg.K,
                  region=region*ureg.um, boundaries = ['p','p','f'], walls = [False,False,True],
                  dipole_cutoff = 30*ureg.um)

In [5]:
sim = mgc.sim(dir_name = "C:/Users/aortiza/Desktop/Test2/",
       timestep = 1e-4*ureg.s, framerate = 30*ureg.Hz, total_time = 10*ureg.s,
       particles = particles, world = world, field = field, output=["x","y","z","mux","muy","muz","fx","fy"])

In [6]:
sim.generate_scripts()
sim.run()

In [7]:
sim.load()
trj = sim.lazy_read[::10]

In [8]:
HTML(mgc.display_animation_direct(sim,trj,start=0,end=60,step=3,speedup=3))

# Set dimers initially

In [9]:
field = mgc.field(magnitude = 7.28*ureg.mT, frequency = 1*ureg.Hz, angle = 27*ureg.degrees,multibody_iter=2)

sim = mgc.sim(dir_name = "C:/Users/aortiza/Desktop/Test2/",file_name="test",
       timestep = 50e-6*ureg.s, framerate = 30*ureg.Hz, total_time = 60*ureg.s,
       particles = particles, world = world, field = field,output=["x","y","z","mux","muy","muz","fx","fy"])

world = mgc.world(particles, temperature = 300*ureg.K,
                  region=region*ureg.um, boundaries = ['p','p','f'], walls = [False,False,True],
                  dipole_cutoff = 10*ureg.um,
                  lj_parameters = [1e-2*ureg.pg*ureg.um**2/ureg.us**2, 2**(-1/6)])

sim.generate_scripts()
sim.run()

In [10]:
sim.load()
trj = sim.lazy_read[::10]
HTML(mgc.display_animation_direct(sim,trj,start=0,end=60,step=1,speedup=3))

In [11]:
frames = trj.index.get_level_values("frame").unique()

In [12]:
particles = mgc.particles(
    trj.loc[idx[frames[-1],:],:].filter(["x","y","z"]).values*ureg.um,
    radius = 1.4*ureg.um,
    susceptibility = 0.4,
    diffusion=0.073*ureg.um**2/ureg.s,
    density = 1000*ureg.kg/ureg.m**3,
    temperature=300*ureg.K)

In [13]:
field = mgc.field(magnitude = 7.28*ureg.mT, frequency = 0*ureg.Hz, angle = 0*ureg.degrees,multibody_iter=2)

sim = mgc.sim(dir_name = "C:/Users/aortiza/Desktop/Test2/",file_name="test",
       timestep = 50e-6*ureg.s, framerate = 30*ureg.Hz, total_time = 60*ureg.s,
       particles = particles, world = world, field = field,output=["x","y","z","mux","muy","muz","fx","fy"])

world = mgc.world(particles, temperature = 300*ureg.K,
                  region=region*ureg.um, boundaries = ['p','p','f'], walls = [False,False,True],
                  dipole_cutoff = 10*ureg.um,
                  lj_parameters = [1e-2*ureg.pg*ureg.um**2/ureg.us**2, 2**(-1/6)])

sim.generate_scripts()
sim.run()

In [14]:
sim.load()
trj = sim.lazy_read[::10]
HTML(mgc.display_animation_direct(sim,trj,start=0,end=60,step=10,speedup=10))

# With no multibody interactions

In [15]:
field = mgc.field(magnitude = 7.28*ureg.mT, frequency = 0*ureg.Hz, angle = 0*ureg.degrees,multibody_iter=0)

sim = mgc.sim(dir_name = "C:/Users/aortiza/Desktop/Test2/",file_name="test",
       timestep = 50e-6*ureg.s, framerate = 30*ureg.Hz, total_time = 60*ureg.s,
       particles = particles, world = world, field = field,output=["x","y","z","mux","muy","muz","fx","fy"])

world = mgc.world(particles, temperature = 300*ureg.K,
                  region=region*ureg.um, boundaries = ['p','p','f'], walls = [False,False,True],
                  dipole_cutoff = 10*ureg.um,
                  lj_parameters = [1e-2*ureg.pg*ureg.um**2/ureg.us**2, 2**(-1/6)])

sim.generate_scripts()
sim.run()

In [16]:
sim.load()
trj = sim.lazy_read[::10]
HTML(mgc.display_animation_direct(sim,trj,start=0,end=60,step=10,speedup=10))